# Scraping Tools



## Reddit

Because whether i like it or not it is an information authority.




### Scrape Links



#### the code

reddit scraper?

TODO: go to reddit and download links and articles from every page in a list.


In [ ]:
const selectDom = importer.import('selenium select')
const getClient = importer.import('selenium client')

async function redditLinks(driver, startPage) {
  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  if(!driver) {
    driver = await getClient()
  }

  try {
    await driver.get(startPage)

    await new Promise(resolve => setTimeout(resolve, 1000))

    let links = await selectDom(driver, [
      '//div[contains(@role, "main")]//div[contains(@class, "link") and not(contains(@class, "linklisting")) and not(contains(@class, "promoted"))]'])

    // TODO: get some special links, get comment count, titles, finally next page
    let results = []
    for(let i = 0; i < links.length; i++) {
      let result = await selectDom(driver, {
        title: './/a[contains(@class, "title")]/text()',
        link: './/a[contains(@class, "title")]/@href',
        time: './/time/@datetime',
        comment: './/a[contains(@class, "comments")]/text()',
      }, links[i])
      results.push(result)
    }

    let next = await selectDom(driver, '//a[contains(@rel, "next")]/@href')

    let objectArray = results.map(r => ({
      title: r.title,
      time: new Date(r.time),
      link: r.link.includes('://') ? r.link : (startPage + (!r.link.startsWith('/') ? '/' : '') + r.link),
      comment: r.comment
    }))
    objectArray.next = next
    return objectArray
  } catch (e) {
    driver.quit()

    throw e
  }
}

module.exports = redditLinks


#### test scraper

test reddit scraper?

In [ ]:
const redditLinks = importer.import('reddit scraper')
const getClient = importer.import('selenium client')

async function testScraper(startPage = 'https://www.reddit.com/r/CollapseSupport+climatechange+collapse+economicCollapse/') {
  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  driver = await getClient()

  let result = await redditLinks(driver, startPage)

  driver.quit()

  return result
}


module.exports = testScraper



### Week/Month Extractor

Extract and entire week/month of links, go to the next page until the articles are older than 1 week/month.



#### the code

reddit month of links?



In [ ]:

const redditLinks = importer.import('reddit scraper')
const getClient = importer.import('selenium client')

async function redditList(driver, startPage, timeSpan = 'week') {
  if(!startPage) {
    startPage = 'https://www.reddit.com/r/CollapseSupport+climatechange+collapse+economicCollapse/'
  }

  if(!startPage.includes('://')) {
    startPage = 'https://www.reddit.com/r/' + startPage
  }

  if(!driver)
    driver = await getClient()

  try {

    let safety = 20

    let weekAgo = new Date(Date.now() - 1000 * 60 * 60 * 24 * 7)
    let monthAgo = new Date(Date.now() - 1000 * 60 * 60 * 24 * 7 * 4.2)

    let finalResult = []

    let result
    let beforeTimeSpan = 0
    do {

      console.log(startPage)

      result = await redditLinks(driver, startPage)

      finalResult = finalResult.concat(result)

      startPage = result.next

      // not yet includes any articles over a month ago
      beforeTimeSpan = result.filter(r => timeSpan == 'month' 
        ? r.time < monthAgo : r.time < weekAgo).length

      safety--

      if(result.next && beforeTimeSpan == 0 && safety > 0) {
        await new Promise(resolve => setTimeout(resolve, 1000))
      }

    } while(result.next && beforeTimeSpan == 0 && safety > 0)

    return finalResult;

  } catch (e) {
    driver.quit()

    throw e
  }
}


module.exports = redditList





### Weekly Summarizer



#### the code

reddit weekly?


In [ ]:

// TODO: connect reddit article extractor with weekly summarizer below.


// TODO: email the results?




## LLM Tools




### Article Extract



#### the code

extract llm article?


In [ ]:
const getClient = importer.import('selenium client')
const llama = importer.import('llm code')
const selectDom = importer.import('selenium select')

// TODO: help me extract this article from html, only return the article in plain text and remove the html:


async function extractArticle(driver, startPage) {
  let llmCode = await llama

  if(!driver)
    driver = await getClient()

  if(!startPage) {
    return
  }

  try {
    await driver.get(startPage)

    let bodyElements = await selectDom(driver, ['//*[text() and not(self::script|self::style|self::form) and not(ancestor::aside|ancestor::nav|ancestor::form)]', './text()'])
    return bodyElements
      .map(t => Array.isArray(t) ? t.join('\n').trim() : t.trim())
      .filter(t => t.length)
      .join('\n')
/*

    let q1 = 'Extract the text content from this HTML page without forming any new HTML code:\n'
      + bodyElements.join('\n') + '\nReturn plain text extracted from the page and nothing else.'

    console.log('User: ' + q1)
    let a1 = await llmCode(q1)
    console.log('AI: ' + a1)

    let code = a1.matchAll(/```(markdown)*\n[\s\S]*?\n```/gi)

    // extract code blocks from response
    let codeBlocks = ''
    for(let match of code) {
      codeBlocks += match[0].replace(/^```(markdown)*\n|\n```$/gi, '') + '\n'
    }
    if(!codeBlocks) {
      console.log('Error, couldn\'t find code in:' + a1)
      return
    }

    return codeBlocks.join('\n')
*/

    } catch (e) {
    driver.quit()

    throw e
  }
}


module.exports = extractArticle



test article extract?


In [ ]:


const extractArticle = importer.import('extract llm article')
const getClient = importer.import('selenium client')

async function testExtractor(startPage) {
  if(!startPage) {
    startPage = 'https://tsakraklides.com/2025/02/05/in-the-age-of-infinite-consumer-choice-the-only-choice-is-collapse/'
  }

  driver = await getClient()

  let result = await extractArticle(driver, startPage)

  driver.quit()

  return result
}


module.exports = testExtractor





### Summarizer



#### the code

summarize llm article?



In [ ]:

const llama = importer.import('create llm session')

// TODO: prompt llm, in one sentence summarize this article. in a whole paragraph summarize this article.

async function summerizeArticle(article) {
  let {llmPrompt} = await llama

  let q1 = 'Summerize this article in a short paragraph:\n' + article + '\nOnly return the summary and nothing else, no explanations.'
  console.log('User: ' + q1)
  let a1 = await llmPrompt(q1)
  console.log('AI: ' + a1)


  let q2 = 'Summerize this article in a single sentence:\n' + article + '\nOnly return the summary and nothing else, no explanations.'
  console.log('User: ' + q2)
  let a2 = await llmPrompt(q2)
  console.log('AI: ' + a2)

  return [a1, a2]
}


module.exports = summerizeArticle



test article summarizer?



In [ ]:
const extractArticle = importer.import('extract llm article')
const getClient = importer.import('selenium client')
const summerizeArticle = importer.import('summarize llm article')

async function testExtractor(startPage) {
  if(!startPage) {
    startPage = 'https://tsakraklides.com/2025/02/05/in-the-age-of-infinite-consumer-choice-the-only-choice-is-collapse/'
  }

  driver = await getClient()

  let result = await extractArticle(driver, startPage)

  driver.quit()

  let summary = await summerizeArticle(result)

  return summary
}


module.exports = testExtractor



### Summarize All



#### the code

summarize all articles?



In [ ]:
const extractArticle = importer.import('extract llm article')
const summerizeArticle = importer.import('summarize llm article')

async function defaultCollector(startPage, selector = '//a[@href]/@href') {
  const selectDom = importer.import('selenium select')
  const getClient = importer.import('selenium client')
  const driver = getClient()

  try {
    await driver.get(startPage)
    await new Promise(resolve => setTimeout(resolve, 1000))
    let links = await selectDom(driver, selector)
    return links.map(l => ({link: l})) // to match reddit post lister
  } catch (e) {
    driver.quit()
    throw e
  }
}


async function summerizeAll(links, selector, startPage) {

  if(!startPage && !links) {
    console.error('No start page or links to summerize.')
    return
  }

  if(!selector && startPage.includes('reddit.com')) {
    selector = importer.import('reddit scraper')
  } else if(!selector) {
    selector = defaultCollector
  } else if(typeof selector == 'string') {
    selector = defaultCollector.bind(null, startPage, selector)
  }

  driver = await getClient()

  try {
    if(!links && startPage) {
      links = await selector(startPage)
    }
    let summaries = []
    for (let i = 0; i < links.length; i++) {
      let article = await extractArticle(driver, links[i])

      let summary = await summerizeArticle(article)
      
      summaries.push(summary)
    }

    return summaries
  } catch (e) {
    driver.quit()
    throw e
  }
}

module.exports = summerizeAll
